In [3]:
import re
import requests
import json

import pymysql

# your username and password go here!
creds = {'user': 'brian', 
         'pswd': 'farley'} 


connection = pymysql.connect(host="atriskwiki.cbprs6vpqcbz.us-west-1.rds.amazonaws.com",
                            user=creds['user'],
                            password=creds['pswd'],
                            port=3306,
                            db="atriskwiki",
                            charset="utf8")

cur = connection.cursor()

In [4]:
monthpart = r"""(?P<month>jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)"""
daypart = r"(?P<day>\d{1,2})\w{0,2}"
yearpart = r"(?P<year>\d{4})"

america = r"(?P<date>{} {},? {})".format(monthpart, daypart, yearpart)
europe = r"(?P<date>{} {} {})".format(daypart, monthpart, yearpart)

print(america)
print(europe)

amdate = re.compile(america, re.I)
eurodate = re.compile(europe, re.I)

(?P<date>(?P<month>jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?) (?P<day>\d{1,2})\w{0,2},? (?P<year>\d{4}))
(?P<date>(?P<day>\d{1,2})\w{0,2} (?P<month>jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?) (?P<year>\d{4}))


In [113]:
epafulltextsURL = "https://storage.googleapis.com/atriskwiki/epa_fulltext.txt"
epafulltexts = requests.get(epafulltextsURL).text.split("\n")
nepafulltextsURL = "https://storage.googleapis.com/atriskwiki/nepa_fulltext.txt"
nepafulltexts = requests.get(nepafulltextsURL).text.split("\n")

In [114]:
extract_num = 666
trial = json.loads(epafulltexts[extract_num])
text = trial[list(trial.keys())[0]]
print(text)

The Palestine Liberation Organization (PLO) (Arabic: منظمة التحرير الفلسطينية‎‎;  Munaẓẓamat at-Taḥrīr al-Filasṭīniyyah ) is an organization founded in 1964 with the purpose of the "liberation of Palestine" through armed struggle. It is recognized as the "sole legitimate representative of the Palestinian people" by over 100 states with which it holds diplomatic relations, and has enjoyed observer status at the United Nations since 1974. The PLO was considered by the United States and Israel to be a terrorist organization until the Madrid Conference in 1991. In 1993, the PLO recognized Israel's right to exist in peace, accepted UN Security Council resolutions 242 and 338, and rejected "violence and terrorism"; in response, Israel officially recognized the PLO as the representative of the Palestinian people.


== FoundingEdit ==
At its first summit meeting in Cairo in 1964, the Arab League initiated the creation of an organization representing the Palestinian people. The Palestinian Nati

In [126]:
eumatches = re.findall(eurodate, text)

In [127]:
eumatches

[('28 May 1964', '28', 'May', '1964'),
 ('2 June 1964', '2', 'June', '1964'),
 ('4 February 1969', '4', 'February', '1969'),
 ('22 November 1974', '22', 'November', '1974'),
 ('12 January 1976', '12', 'January', '1976'),
 ('2 April 1986', '2', 'April', '1986'),
 ('7 July 1998', '7', 'July', '1998'),
 ('29 November 2012', '29', 'November', '2012'),
 ('20 July 2010', '20', 'July', '2010'),
 ('9 June 1974', '9', 'June', '1974'),
 ('20 August 1993', '20', 'August', '1993'),
 ('13 September 1993', '13', 'September', '1993'),
 ('20 January 1976', '20', 'January', '1976'),
 ('1 October 1985', '1', 'October', '1985'),
 ('2 June 1964', '2', 'June', '1964'),
 ('24 December 1967', '24', 'December', '1967'),
 ('24 December 1967', '24', 'December', '1967'),
 ('2 February 1969', '2', 'February', '1969'),
 ('4 February 1969', '4', 'February', '1969'),
 ('11 November 2004', '11', 'November', '2004'),
 ('29 October 2004', '29', 'October', '2004'),
 ('11 November 2004', '11', 'November', '2004'),
 ('8 J

In [129]:
context = text.partition(eumatches[0][0])
context

('The Palestine Liberation Organization (PLO) (Arabic: منظمة التحرير الفلسطينية\u200e\u200e;  Munaẓẓamat at-Taḥrīr al-Filasṭīniyyah ) is an organization founded in 1964 with the purpose of the "liberation of Palestine" through armed struggle. It is recognized as the "sole legitimate representative of the Palestinian people" by over 100 states with which it holds diplomatic relations, and has enjoyed observer status at the United Nations since 1974. The PLO was considered by the United States and Israel to be a terrorist organization until the Madrid Conference in 1991. In 1993, the PLO recognized Israel\'s right to exist in peace, accepted UN Security Council resolutions 242 and 338, and rejected "violence and terrorism"; in response, Israel officially recognized the PLO as the representative of the Palestinian people.\n\n\n== FoundingEdit ==\nAt its first summit meeting in Cairo in 1964, the Arab League initiated the creation of an organization representing the Palestinian people. The

In [5]:
def GetDateInContext(dateString, ArticleText, length=150):
    partitioned_str = ArticleText.partition(dateString)
    up = str(partitioned_str[0])
    date = str(partitioned_str[1])
    down = str(partitioned_str[2])
    
    getUp = min(length, len(up))
    getDown = min(length, len(down))
    noLetters = re.compile("[^A-Za-z]+")
    return(re.sub(noLetters, " ", up[-getUp:]),date,re.sub(noLetters," ",down[:getDown]))

In [135]:
GetDateInContext(context)

('tion representing the Palestinian people. The Palestinian National Council convened in Jerusalem on ',
 '28 May 1964',
 '. Concluding this meeting the PLO was founded on 2 June 1964. Its stated goal was the "liberation of')

['January 18, 2008']

In [6]:
import datetime
monthhash = {'jan': 1,
            'feb': 2,
            'mar': 3,
            'apr': 4,
            'may': 5,
            'jun': 6,
            'jul': 7,
            'aug': 8,
            'sep': 9,
            'oct': 10,
            'nov': 11,
            'dec': 12}

def GetDatesFromArticle(articleText,listOfRegex=[amdate,eurodate]):
    datesInContext = dict()
    datesForMath = []
    for pattern in listOfRegex:
        for m in re.finditer(pattern, articleText):
            foundDate = m.group('date')
            foundDay = int(m.group('day'))
            foundMonth = monthhash[str(m.group('month')).lower()[:3]]
            foundYear = int(m.group('year'))
            try:
                d = datetime.date(year=foundYear,month=foundMonth,day=foundDay)
                formattedDate = "{}-{:02d}-{:02d}".format(foundYear,foundMonth,foundDay)
                before, date, after = GetDateInContext(foundDate, articleText)
                datesInContext[formattedDate] = {'upstream':before,
                                                 'date':date,
                                                 'downstream':after}
                datesForMath.append(d)
            except ValueError as v:
                print("{}: {}".format(foundDate,v))
    
    metaDate = {'medianDate':None,
                'mostRecentDate':None}
    if datesForMath:
        metaDate['mostRecentDate'] = max(datesForMath).strftime("%Y-%m-%d")
        dateCount = len(datesForMath)
        datesForMath.sort()
        if dateCount == 0:
            medianDate = datesForMath[0]
        elif (dateCount % 2 == 1):
            medianIndex = (dateCount - 1) // 2
            medianDate = datesForMath[medianIndex]
        else:
            medianIndices = (dateCount//2, dateCount//2 - 1)
            medianDate = datesForMath[medianIndices[0]] + (datesForMath[medianIndices[1]] - datesForMath[medianIndices[0]]) / 2
        metaDate['medianDate'] = medianDate.strftime("%Y-%m-%d")
    datesInContext['meta'] = metaDate
    return datesInContext


In [162]:
GetDatesFromArticle(text)

{'1964-05-28': {'date': '28 May 1964',
  'downstream': ' Concluding this meeting the PLO was founded on June Its stated goal was the liberation of Palestine through armed struggle IdeologyE',
  'upstream': ' Arab League initiated the creation of an organization representing the Palestinian people The Palestinian National Council convened in Jerusalem on '},
 '1964-06-02': {'date': '2 June 1964',
  'downstream': ' Its stated goal was the liberation of Palestine through armed struggle IdeologyEdit The ideology of the PLO was formulated in the foundin',
  'upstream': 'senting the Palestinian people The Palestinian National Council convened in Jerusalem on May Concluding this meeting the PLO was founded on '},
 '1967-12-24': {'date': '24 December 1967',
  'downstream': ' Yahya Hammuda December February Yasser Arafat Abu Amar February November in exile in Jordan to April ',
  'upstream': 'he Popular Front for the Liberation of Palestine General Command PFLP GC Executive Committee ChairmenEd

In [171]:
ftList = [nepafulltexts, epafulltexts]
contextRecords = []
metaRecords = []
for corpus in ftList:
    for ft in corpus:
        try:
            article_ID = list(json.loads(ft).keys())[0]
            article_text = list(json.loads(ft).values())[0]
            dates = GetDatesFromArticle(article_text)
            for record in dates.keys():
                if record == 'meta':
                    metaRecords.append({'page_id':article_ID,
                                        'medianDate':dates[record]['medianDate'],
                                        'mostRecentDate':dates[record]['mostRecentDate']})
                else:
                    contextRecords.append({'page_id':article_ID,
                                           'ft_date':record,
                                           'prev_context':dates[record]['upstream'],
                                           'text_date':dates[record]['date'],
                                           'next_context':dates[record]['downstream']})

        except:
            pass


March 91, 1877: day is out of range for month
4449 May 1960: day is out of range for month
4502 Sep 1960: day is out of range for month
4597 Mar 1961: day is out of range for month
4625 Apr 1961: day is out of range for month
323 Aug 1961: day is out of range for month
514 Nov 1964: day is out of range for month
361 Jan 1962: day is out of range for month
29 February 2003: day is out of range for month
November 0f 1900: day is out of range for month
75 January 1880: day is out of range for month
76 June 1880: day is out of range for month
77 June 1881: day is out of range for month
78 June 1882: day is out of range for month
79 January 1883: day is out of range for month
95 January 1891: day is out of range for month
96 June 1891: day is out of range for month
97 June 1892: day is out of range for month
31 June 1821: day is out of range for month
June 31, 2005: day is out of range for month
5135 February 2006: day is out of range for month
80 March 1985: day is out of range for month
J

In [172]:
metaRecords[:5]

[{'medianDate': '2010-10-21',
  'mostRecentDate': '2015-10-03',
  'page_id': '32860140'},
 {'medianDate': None, 'mostRecentDate': None, 'page_id': '24542042'},
 {'medianDate': '2013-04-05',
  'mostRecentDate': '2016-06-14',
  'page_id': '37501742'},
 {'medianDate': '2011-03-18',
  'mostRecentDate': '2016-06-30',
  'page_id': '21019435'},
 {'medianDate': '1948-04-03',
  'mostRecentDate': '1948-04-03',
  'page_id': '36270733'}]

In [173]:
contextRecords[:5]

[{'ft_date': '2015-10-03',
  'next_context': ' Personal life Nash became a father in September to a little girl name LeKenleigh Nash Awards and honors Newcomer the Yea',
  'page_id': '32860140',
  'prev_context': 'with the Fukushima Firebonds of the Japanese bj league and made his professional debut in the season opening match against the Yokohama B Corsairs on ',
  'text_date': '3 October 2015'},
 {'ft_date': '2010-10-21',
  'next_context': ' Nash committed and signed to play basketball with Oklahoma State University after visiting the campus on August of the same year Oklahoma State ',
  'page_id': '32860140',
  'prev_context': ' and the player by Scout com In the ESPNU basketball rankings Nash was the rd ranked small forward in his class College career On ',
  'text_date': 'October 21, 2010'},
 {'ft_date': '1992-06-30',
  'next_context': ' is an American professional basketball player for the Fukushima Firebonds of the Japanese bj league He attended Lincoln High School in Dallas Tex

In [7]:
import os
import datetime
path = "ArticleFulltexts/"
metaRecords = []
contextRecords = []
for filename in os.listdir(path):
    if filename == "epa_fulltext.txt":
        pass
    else:
        with open(path+filename,"r") as f:
            now = datetime.datetime.now().time()
            print("[{:02d}:{:02d}:{:02d}] opening {}".format(now.hour,now.minute,now.second,filename))
            for ix,ft in enumerate(f):
                if (ix + 1) % 1000 == 0:
                    now = datetime.datetime.now().time()
                    print("[{:02d}:{:02d}:{:02d}] parsing record {}".format(now.hour,now.minute,now.second,ix+1))
                try:
                    article_ID = list(json.loads(ft).keys())[0]
                    article_text = list(json.loads(ft).values())[0]
                    dates = GetDatesFromArticle(article_text)
                    for record in dates.keys():
                        if record == 'meta':
                            metaRecords.append({'page_id':article_ID,
                                                'medianDate':dates[record]['medianDate'],
                                                'mostRecentDate':dates[record]['mostRecentDate']})
                        else:
                            contextRecords.append({'page_id':article_ID,
                                                   'ft_date':record,
                                                   'prev_context':dates[record]['upstream'],
                                                   'text_date':dates[record]['date'],
                                                   'next_context':dates[record]['downstream']})

                except:
                    pass



[12:55:27] opening 071816_fulltext_batch1.txt
[12:55:33] parsing record 1000
[12:55:37] parsing record 2000
[12:55:40] parsing record 3000
[12:55:43] parsing record 4000
February 31, 2009: day is out of range for month
[12:55:45] parsing record 5000
[12:55:47] parsing record 6000
[12:55:49] parsing record 7000
[12:55:51] parsing record 8000
[12:55:53] parsing record 9000
634 Nov 2006: day is out of range for month
[12:55:54] opening 071816_fulltext_batch2.txt
32 July 1968: day is out of range for month
[12:56:00] parsing record 1000
31 September 2013: day is out of range for month
88 May 1989: day is out of range for month
[12:56:03] parsing record 2000
September 31, 1966: day is out of range for month
June 31, 2001: day is out of range for month
[12:56:06] parsing record 3000
[12:56:09] parsing record 4000
September 31, 2009: day is out of range for month
[12:56:11] parsing record 5000
[12:56:13] parsing record 6000
31 June 2010: day is out of range for month
[12:56:15] parsing record

In [176]:
cur.execute("SET autocommit=0")

0

In [177]:
createCE = '''CREATE TABLE IF NOT EXISTS current_events (
              page_id INT(8) UNSIGNED NOT NULL,
              medianDate DATE,
              mostRecentDate DATE,
              PRIMARY KEY (page_id)
              ) DEFAULT CHARACTER SET utf8'''
cur.execute(createCE)
connection.commit()

In [8]:
insertCE = "INSERT IGNORE INTO current_events({}) VALUES ({})"
cols = ",".join(list(metaRecords[0].keys()))
val_format = ",".join(['%s']*len(list(metaRecords[0].keys())))
formatCE = insertCE.format(cols, val_format)
formatCE

'INSERT IGNORE INTO current_events(page_id,medianDate,mostRecentDate) VALUES (%s,%s,%s)'

In [9]:
for ix,record in enumerate(metaRecords):
    if (ix+1) % 5000 == 0:
        now = datetime.datetime.now().time()
        print("[{:02d}:{:02d}:{:02d}] inserting record {} of {}".format(now.hour,now.minute,now.second,ix+1,len(metaRecords)))
    values = list(record.values())
    cur.execute(formatCE, values)
    
connection.commit()

[13:05:50] inserting record 5000 of 75771
[13:09:11] inserting record 10000 of 75771
[13:14:09] inserting record 15000 of 75771
[13:15:56] inserting record 20000 of 75771
[13:17:03] inserting record 25000 of 75771
[13:18:28] inserting record 30000 of 75771
[13:19:33] inserting record 35000 of 75771
[13:20:35] inserting record 40000 of 75771
[13:21:29] inserting record 45000 of 75771
[13:22:25] inserting record 50000 of 75771
[13:23:28] inserting record 55000 of 75771
[13:24:36] inserting record 60000 of 75771
[13:25:49] inserting record 65000 of 75771
[13:27:12] inserting record 70000 of 75771
[13:28:19] inserting record 75000 of 75771


In [180]:
createDC = '''CREATE TABLE IF NOT EXISTS date_context (
              page_id INT(8) UNSIGNED NOT NULL,
              ft_date DATE NOT NULL,
              prev_context TEXT,
              text_date VARCHAR(20),
              next_context TEXT,
              PRIMARY KEY (page_id, ft_date)
              ) DEFAULT CHARACTER SET utf8'''
cur.execute(createDC)
connection.commit()

In [10]:
def BatchInsertRecordsFromListOfDicts(table, records, cursor=cur, connection=connection):
    insert = "INSERT IGNORE INTO {}({}) VALUES ({})"
    cols = ",".join(list(records[0].keys()))
    val_format = ",".join(['%s']*len(list(records[0].keys())))
    formatI = insert.format(table, cols, val_format)
    totRecords = len(records)
    now = datetime.datetime.now().time()
    print("[{:02d}:{:02d}:{:02d}] {} records to insert".format(now.hour,now.minute,now.second,totRecords))
    for ix,record in enumerate(records):
        if (ix + 1) % 5000 == 0:
            now = datetime.datetime.now().time()
            print("[{:02d}:{:02d}:{:02d}] {} of {} records processed".format(now.hour,now.minute,now.second,ix + 1,totRecords))
        values = list(record.values())
        cur.execute(formatI,values)
    print("Finished; committing transaction.")
    connection.commit()
    print("Done!")

In [11]:
BatchInsertRecordsFromListOfDicts("date_context",contextRecords)

[13:31:00] 222019 records to insert
[13:31:57] 5000 of 222019 records processed
[13:33:00] 10000 of 222019 records processed
[13:34:01] 15000 of 222019 records processed
[13:35:06] 20000 of 222019 records processed
[13:36:13] 25000 of 222019 records processed
[13:37:16] 30000 of 222019 records processed
[13:38:30] 35000 of 222019 records processed
[13:39:38] 40000 of 222019 records processed
[13:41:06] 45000 of 222019 records processed
[13:42:28] 50000 of 222019 records processed
[13:44:01] 55000 of 222019 records processed
[13:45:04] 60000 of 222019 records processed
[13:46:00] 65000 of 222019 records processed
[13:46:56] 70000 of 222019 records processed
[13:47:54] 75000 of 222019 records processed
[13:48:56] 80000 of 222019 records processed
[13:49:57] 85000 of 222019 records processed
[13:51:03] 90000 of 222019 records processed
[13:52:16] 95000 of 222019 records processed
[13:53:59] 100000 of 222019 records processed
[13:55:11] 105000 of 222019 records processed
[13:56:33] 110000 